# 02 - HuggingFace Tokenizers: Comparing Production Tokenizers on Legal Text

In the previous notebook we built BPE from scratch. In practice, LLMs use
highly optimized tokenizers trained on massive corpora. This notebook explores
the tokenizers behind GPT-2 and LLaMA-2 using the HuggingFace `transformers`
and `tokenizers` libraries.

We will:
1. Load pre-trained tokenizers and see how they work
2. Tokenize legal text and compare token counts across models
3. Visualize token boundaries to see where each tokenizer splits
4. Compute **token fertility** -- the average number of tokens per word --
   for legal vs general text
5. Train a custom tokenizer on a legal corpus and compare it against GPT-2

## Setup

In [ ]:
# Install dependencies (uncomment if needed)
# %pip install transformers tokenizers matplotlib

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
from transformers import AutoTokenizer

## 1. Loading Pre-trained Tokenizers

HuggingFace provides tokenizers for every model on the Hub. We load two
widely used tokenizers:

- **GPT-2** -- BPE with 50,257 tokens, trained on WebText
- **LLaMA-2** -- BPE with 32,000 tokens, trained on a larger and more
  diverse corpus

These two tokenizers were trained on different data with different vocabulary
sizes, which leads to measurably different behavior on domain-specific text.

In [ ]:
# Load tokenizers
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
llama_tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    # If you don't have access to LLaMA-2, you can use an open alternative:
    # "NousResearch/Llama-2-7b-hf"
)

print(f"GPT-2 vocab size:  {gpt2_tokenizer.vocab_size:,}")
print(f"LLaMA-2 vocab size: {llama_tokenizer.vocab_size:,}")

In [ ]:
# Basic tokenization demonstration
sample = "The court granted summary judgment."

for name, tokenizer in [("GPT-2", gpt2_tokenizer), ("LLaMA-2", llama_tokenizer)]:
    token_ids = tokenizer.encode(sample)
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    decoded = tokenizer.decode(token_ids)
    print(f"\n{name}:")
    print(f"  Token IDs: {token_ids}")
    print(f"  Tokens:    {tokens}")
    print(f"  Decoded:   {decoded}")
    print(f"  Count:     {len(token_ids)} tokens")

## 2. Tokenizing Legal Text

Let's load our court opinions dataset and see how these tokenizers handle
real legal text.

In [ ]:
DATA_PATH = Path("../../datasets/sample/court_opinions.jsonl")

opinions = []
with open(DATA_PATH) as f:
    for line in f:
        opinions.append(json.loads(line))

print(f"Loaded {len(opinions)} court opinions")
for op in opinions:
    print(f"  - {op['case_name'][:60]}...")

In [ ]:
# Key example: tokenizing a legal citation
citation_examples = [
    "42 U.S.C. \u00a7 1983",
    "Anderson v. Liberty Lobby, Inc., 477 U.S. 242, 255 (1986)",
    "35 U.S.C. section 271",
    "20 U.S.C. sections 1400-1482",
    "Federal Rule of Civil Procedure 12(b)(6)",
]

print(f"{'Citation':<55} {'GPT-2':>6} {'LLaMA':>6}")
print("-" * 70)
for citation in citation_examples:
    gpt2_count = len(gpt2_tokenizer.encode(citation))
    llama_count = len(llama_tokenizer.encode(citation))
    print(f"{citation:<55} {gpt2_count:>6} {llama_count:>6}")

In [ ]:
# Detailed breakdown of "42 U.S.C. \u00a7 1983"
example = "42 U.S.C. \u00a7 1983"
print(f"Text: {example}\n")

for name, tokenizer in [("GPT-2", gpt2_tokenizer), ("LLaMA-2", llama_tokenizer)]:
    ids = tokenizer.encode(example)
    tokens = tokenizer.convert_ids_to_tokens(ids)
    print(f"{name} ({len(ids)} tokens):")
    for token_id, token in zip(ids, tokens):
        print(f"  ID {token_id:>6}  ->  {repr(token)}")
    print()

In [ ]:
# Full opinion token count comparison
print(f"{'Case Name':<55} {'Words':>6} {'GPT-2':>7} {'LLaMA':>7} {'Ratio':>7}")
print("-" * 85)

for opinion in opinions:
    text = opinion["text"]
    word_count = len(text.split())
    gpt2_count = len(gpt2_tokenizer.encode(text))
    llama_count = len(llama_tokenizer.encode(text))
    # Ratio: how many more tokens does GPT-2 need vs LLaMA?
    ratio = gpt2_count / llama_count if llama_count > 0 else 0
    name = opinion["case_name"][:53]
    print(f"{name:<55} {word_count:>6} {gpt2_count:>7} {llama_count:>7} {ratio:>7.2f}")

## 3. Visualizing Token Boundaries

To see *where* each tokenizer splits, we write a function that marks the
boundaries between tokens. This makes it easy to spot differences in how
tokenizers segment the same text.

In [ ]:
def show_token_boundaries(text: str, tokenizer, name: str = "") -> str:
    """Show token boundaries using bracket notation.

    Each token is wrapped in brackets: [token1][token2][token3]
    This makes it easy to see where the tokenizer splits the text.
    """
    token_ids = tokenizer.encode(text)
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    # Clean up token representations for display
    display_tokens = []
    for token in tokens:
        # GPT-2 uses \u0120 prefix for space, LLaMA uses \u2581
        cleaned = token.replace("\u0120", " ").replace("\u2581", " ")
        display_tokens.append(cleaned)
    bracketed = "".join(f"[{t}]" for t in display_tokens)
    if name:
        return f"{name} ({len(tokens)} tokens):\n  {bracketed}"
    return bracketed

In [ ]:
legal_passages = [
    "42 U.S.C. \u00a7 1983",
    "The district court's grant of summary judgment",
    "Anderson v. Liberty Lobby, Inc., 477 U.S. 242, 255 (1986)",
    "prima facie case of discrimination",
    "National Pollutant Discharge Elimination System",
]

for passage in legal_passages:
    print(f"Text: {passage}")
    print(show_token_boundaries(passage, gpt2_tokenizer, "GPT-2"))
    print(show_token_boundaries(passage, llama_tokenizer, "LLaMA-2"))
    print()

In [ ]:
# Visualize a longer passage
long_passage = opinions[0]["text"][:300]
print(f"Passage (first 300 chars of {opinions[0]['case_name']}):\n")
print(f"{long_passage}\n")
print(show_token_boundaries(long_passage, gpt2_tokenizer, "GPT-2"))
print()
print(show_token_boundaries(long_passage, llama_tokenizer, "LLaMA-2"))

## 4. Token Fertility Analysis

**Token fertility** is the average number of tokens produced per whitespace
word. A fertility of 1.0 means each word maps to exactly one token. Values
above 1.0 mean words are being split into subword pieces.

Higher fertility on legal text compared to general text means:
- Legal documents consume more of the context window
- API costs are proportionally higher
- The model must compose meaning from more fragmented representations

In [ ]:
def compute_fertility(text: str, tokenizer) -> float:
    """Compute token fertility: average tokens per whitespace word."""
    word_count = len(text.split())
    token_count = len(tokenizer.encode(text))
    if word_count == 0:
        return 0.0
    return token_count / word_count

In [ ]:
# Legal text: from court opinions dataset
legal_text = " ".join(op["text"] for op in opinions)

# General English text: a plain-language paragraph for comparison
general_text = (
    "The weather was warm and sunny on Saturday morning. Sarah decided to go "
    "for a walk in the park near her house. She brought her dog Max along for "
    "the exercise. The park had tall oak trees and a small pond where ducks "
    "were swimming. Children were playing on the swings and slides while their "
    "parents watched from nearby benches. After walking for about an hour she "
    "stopped at the coffee shop on the corner and ordered a latte. The barista "
    "recognized her and asked about her weekend plans. She mentioned that she "
    "was thinking about visiting the new bookstore that had just opened downtown. "
    "It was a pleasant morning and she felt grateful for the simple joy of a "
    "good walk on a beautiful day. When she got home she made breakfast and "
    "read the newspaper while Max napped on the couch beside her."
)

print(f"{'Corpus':<15} {'Words':>7} {'GPT-2 tokens':>13} {'GPT-2 fert.':>12} "
      f"{'LLaMA tokens':>13} {'LLaMA fert.':>12}")
print("-" * 75)

for corpus_name, text in [("Legal", legal_text), ("General", general_text)]:
    word_count = len(text.split())
    gpt2_tokens = len(gpt2_tokenizer.encode(text))
    llama_tokens = len(llama_tokenizer.encode(text))
    gpt2_fert = compute_fertility(text, gpt2_tokenizer)
    llama_fert = compute_fertility(text, llama_tokenizer)
    print(f"{corpus_name:<15} {word_count:>7} {gpt2_tokens:>13} {gpt2_fert:>12.3f} "
          f"{llama_tokens:>13} {llama_fert:>12.3f}")

In [ ]:
# Per-opinion fertility comparison
gpt2_fertilities = []
llama_fertilities = []

for op in opinions:
    gpt2_fertilities.append(compute_fertility(op["text"], gpt2_tokenizer))
    llama_fertilities.append(compute_fertility(op["text"], llama_tokenizer))

general_gpt2_fert = compute_fertility(general_text, gpt2_tokenizer)
general_llama_fert = compute_fertility(general_text, llama_tokenizer)

fig, ax = plt.subplots(figsize=(10, 5))
x_labels = [op["case_name"].split(" v. ")[0][:15] for op in opinions] + ["General\nEnglish"]
x = range(len(x_labels))

gpt2_vals = gpt2_fertilities + [general_gpt2_fert]
llama_vals = llama_fertilities + [general_llama_fert]

width = 0.35
bars1 = ax.bar([i - width / 2 for i in x], gpt2_vals, width, label="GPT-2", color="#2563eb")
bars2 = ax.bar([i + width / 2 for i in x], llama_vals, width, label="LLaMA-2", color="#dc2626")

ax.set_ylabel("Token Fertility (tokens per word)")
ax.set_title("Token Fertility: Legal Opinions vs General English")
ax.set_xticks(list(x))
ax.set_xticklabels(x_labels, rotation=30, ha="right")
ax.legend()
ax.axhline(y=1.0, color="gray", linestyle="--", alpha=0.5, label="1:1 ratio")
ax.grid(True, alpha=0.3, axis="y")
plt.tight_layout()
plt.show()

In [ ]:
# Cost implications
legal_fert_gpt2 = sum(gpt2_fertilities) / len(gpt2_fertilities)
overhead = (legal_fert_gpt2 - general_gpt2_fert) / general_gpt2_fert * 100

print(f"Average GPT-2 fertility on legal text:   {legal_fert_gpt2:.3f}")
print(f"GPT-2 fertility on general English:      {general_gpt2_fert:.3f}")
print(f"Legal text token overhead:               {overhead:+.1f}%")
print()
print("Cost implication at $0.01 per 1K tokens:")
print(f"  A 10,000-word legal document produces ~{int(10000 * legal_fert_gpt2):,} tokens")
print(f"  A 10,000-word general document produces ~{int(10000 * general_gpt2_fert):,} tokens")
print(f"  Extra cost per legal document: ${10000 * (legal_fert_gpt2 - general_gpt2_fert) * 0.01 / 1000:.4f}")

## 5. Training a Custom Tokenizer

If legal text tokenizes poorly with general-purpose tokenizers, can we do
better? The HuggingFace `tokenizers` library provides fast, production-quality
BPE training. Let's train a custom tokenizer on our legal corpus and compare
it against GPT-2.

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [ ]:
# Prepare training data -- write the legal corpus to a temporary file
# (the tokenizers library trains from files for efficiency)
import tempfile

corpus_path = Path(tempfile.mktemp(suffix=".txt"))
with open(corpus_path, "w") as f:
    for opinion in opinions:
        f.write(opinion["text"] + "\n")

# Also add legislation for more domain coverage
legislation_path = Path("../../datasets/sample/legislation.jsonl")
if legislation_path.exists():
    with open(legislation_path) as f_in, open(corpus_path, "a") as f_out:
        for line in f_in:
            record = json.loads(line)
            f_out.write(record["text"] + "\n")

print(f"Training corpus written to: {corpus_path}")
print(f"Corpus size: {corpus_path.stat().st_size:,} bytes")

In [ ]:
# Train a BPE tokenizer with a vocabulary size comparable to GPT-2
legal_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
legal_tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(
    vocab_size=10000,  # Smaller vocab since our corpus is small
    special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"],
    min_frequency=2,
    show_progress=True,
)

legal_tokenizer.train([str(corpus_path)], trainer)
print(f"Trained legal tokenizer with {legal_tokenizer.get_vocab_size()} tokens")

In [ ]:
# Compare vocabularies: find legal terms in each tokenizer's vocabulary
legal_vocab = legal_tokenizer.get_vocab()
gpt2_vocab = gpt2_tokenizer.get_vocab()

legal_terms_to_check = [
    "court", "plaintiff", "defendant", "judgment", "injunction",
    "statute", "amend", "jurisdiction", "certiorari", "appellant",
    "appellee", "arbitration", "deposition", "affidavit",
    "subpoena", "tort", "negligence", "liability",
]

print(f"{'Term':<20} {'In Legal Vocab':>15} {'In GPT-2 Vocab':>15}")
print("-" * 52)
for term in legal_terms_to_check:
    in_legal = term in legal_vocab
    in_gpt2 = term in gpt2_vocab or f"\u0120{term}" in gpt2_vocab
    print(f"{term:<20} {'yes':>15} {'yes' if in_gpt2 else 'no':>15}")

In [ ]:
# Compare tokenization of legal text
comparison_texts = [
    "42 U.S.C. \u00a7 1983",
    "summary judgment de novo",
    "prima facie case of discrimination",
    "National Pollutant Discharge Elimination System",
    "per- and polyfluoroalkyl substances",
]

print(f"{'Text':<50} {'GPT-2':>6} {'Legal':>6}")
print("-" * 65)
for text in comparison_texts:
    gpt2_count = len(gpt2_tokenizer.encode(text))
    legal_enc = legal_tokenizer.encode(text)
    legal_count = len(legal_enc.ids)
    print(f"{text:<50} {gpt2_count:>6} {legal_count:>6}")

In [ ]:
# Show how the custom tokenizer segments legal text differently
example = "The district court granted summary judgment in favor of the defendant"

gpt2_ids = gpt2_tokenizer.encode(example)
gpt2_tokens = gpt2_tokenizer.convert_ids_to_tokens(gpt2_ids)
legal_enc = legal_tokenizer.encode(example)
legal_tokens = legal_enc.tokens

print(f"Text: {example}\n")
print(f"GPT-2 ({len(gpt2_tokens)} tokens):")
for t in gpt2_tokens:
    display = t.replace("\u0120", " ")
    print(f"  [{display}]", end="")
print()

print(f"\nLegal ({len(legal_tokens)} tokens):")
for t in legal_tokens:
    print(f"  [{t}]", end="")
print()

In [ ]:
# Clean up temporary file
corpus_path.unlink(missing_ok=True)
print("Cleaned up temporary training corpus.")

## Exercises

### Exercise (a): Fertility by Legal Document Type

Compute the token fertility for different types of legal text:

1. **Statutes** -- Load `datasets/sample/legislation.jsonl` and compute
   fertility for each piece of legislation.
2. **Court opinions** -- Use the court opinions from this notebook.
3. **Contracts** -- Write or find a sample contract clause (e.g., an
   indemnification provision) and compute its fertility.

Compare across tokenizers. Which type of legal text has the highest fertility?
Why might that be? Consider:
- Statutory language uses defined terms and cross-references heavily
- Court opinions include case citations and procedural terminology
- Contracts use boilerplate phrasing and nested conditional clauses

### Exercise (b): Cost Estimation

A legal technology company processes 1,000 court opinions per day, each
averaging 5,000 words. Using an API priced at $0.01 per 1,000 tokens:

1. Compute the daily token count and daily cost using GPT-2's tokenizer
   fertility.
2. Estimate how much could be saved if a custom legal tokenizer reduced
   fertility by 10%, 15%, or 20%.
3. Over a year (250 working days), what is the total savings at each
   reduction level?

Present your results in a table. At what fertility reduction does the cost
savings justify the engineering effort of training and deploying a custom
tokenizer?